# Micrograd
This is a micrograd engine. Micrograd is a small autograd engine. An autograd engine is used for automatic differentiation in tensors and gradients. Micrograd is essentially a small version of that. We shall start by importing all necessary libraries. Even though we are importing some libraries, the code for micrograd will be written from scratch. Later on, we might test out the autograd engine on PyTorch. The libraries we import are math and random.

In [1]:
import math
import random

## Value Objects
A value object is a node in a computation graph here. Mathematically, one can think of it as simply a number. However, for the development of our micrograd architecture, we are to look at value objects as nodes in a computation graph. Here, we define functions for initialization, reproduction, addition, multliplication, exponentiation, reverse multiplication, true division, hyperbolic tangent function, exponentiation function and backward gradient computation. Apart from the hyperbolic tangent, exponentiation function and backward gradient computation, all other functions are special functions for Python. In the backward gradient function, we perform a topological sorting algorithm. This way, the computation graph gets arranged in the form of a linear chain, an array or simply a tensor. This helps in leveraging the GPU resources(which we don't have yet) and makes automatic gradient computation extremely convenient.

In [2]:
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda : None
        self._prev = set(_children)
        self._op = _op
    def __repr__(self):
        return f"Value(data={self.data})"
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f"**{other}")
        def _backward():
            self.grad += other * self.data ** (other - 1) * out.grad # exercise :)
        out._backward = _backward
        return out
    def __rmul__(self, other): # other * self
        return self * other
    def __truediv__(self, other): # self / other
        return self * other ** -1
    def __neg__(self): # - self
        return self * - 1
    def __sub__(self, other): # self - other
        return self + (- other)
    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self, ), 'tanh')
        def _backward():
            self.grad += (1 - t ** 2) * out.grad
        out._backward = _backward
        return out
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.data * out.grad # ??? exercise :)
        out._backward = _backward
        return out
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

## Layers
Layers in a computation graph are levels of depth in which one particular node is situated. A generic computation graph has an input layer, an output layer and hidden layers in between. The weights of the net reside between the layers. Mathematically, one can express the weights between two consecutive layers in the form of a matrix. As said earlier, a matrix is a two dimensional tensor. The goal of training a computation graph is to optimize these weight matrices and bias vectors to minimize a loss function.

In [3]:
class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
    def __call__(self, x):
        # w * x + b
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    def parameters(self):
        return self.w + [self.b]
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

# Definition
Now we define a computation graph. This one takes in three inputs and provides one output. It has three layers. The first two hidden layers consist of four neurons each and the the output layer, as said before, has one neuron. Mathematically, one can think of a computation graph as a highly abstract, non-linear transformation from an input space to an output space. $\\ ComputationGraph: \mathbb{R}^i \rightarrow \mathbb{R}^o$

In [4]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.34077904978630646)

In [5]:
n.parameters()

[Value(data=0.3409343871516124),
 Value(data=-0.06058318243490701),
 Value(data=-0.4441141795796997),
 Value(data=-0.6097174580145774),
 Value(data=-0.9662445362517325),
 Value(data=-0.2509148473917726),
 Value(data=-0.21027110652128989),
 Value(data=-0.4932680280729833),
 Value(data=-0.6464314457919778),
 Value(data=-0.9304389346354631),
 Value(data=-0.11727312704124149),
 Value(data=0.20275808745023216),
 Value(data=0.0332832427753742),
 Value(data=-0.46097424848899693),
 Value(data=-0.4974206561479164),
 Value(data=-0.4334330845091312),
 Value(data=-0.15350402567124877),
 Value(data=0.6616531611429193),
 Value(data=-0.18472958038807175),
 Value(data=-0.6461427288904391),
 Value(data=0.3499599277802974),
 Value(data=0.6206098125762132),
 Value(data=0.29633518093890254),
 Value(data=0.7335312948549997),
 Value(data=0.9595147488635791),
 Value(data=-0.7651786361425401),
 Value(data=0.6845235555215115),
 Value(data=-0.21742264746237905),
 Value(data=-0.8916808711473037),
 Value(data=0.8

In [6]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

## Activation
Let us consider a node in a computation graph. It takes inputs, scales them by their corresponding weights, adds them up, adds a bias, passes the weighted sum to an activation function and returns the result. An activation function is highly relevant here because without one, a model always behaves like a linear model regardless of the number of hidden layers. The activation function used here is the hyperbolic tangent function. Its range is the closed interval from - 1 to 1, or $[-1, 1]$.

$tanh: \mathbb{R} \rightarrow [-1, 1] \hspace{3em} tanh(t) = \frac{e^{t} - e^{-t}}{e^{t} + e^{-t}} \hspace{1em} \forall t \in \mathbb{R} \hspace{3em} tanh'(t) = 1 - tanh(t)^{2}$

In [7]:
for k in range(20):
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum(((yout - ygt) ** 2 for ygt, yout in zip(ys, ypred)), Value(0.0))
    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    print(f"Epoch: {k}, Loss: {loss}")

Epoch: 0, Loss: Value(data=5.983738136954173)
Epoch: 1, Loss: Value(data=4.658961225912644)
Epoch: 2, Loss: Value(data=3.7316404410036905)
Epoch: 3, Loss: Value(data=3.0533340575654444)
Epoch: 4, Loss: Value(data=2.6203498092356794)
Epoch: 5, Loss: Value(data=2.274704760423538)
Epoch: 6, Loss: Value(data=1.9248632192964452)
Epoch: 7, Loss: Value(data=1.516975615195223)
Epoch: 8, Loss: Value(data=1.0785238636176495)
Epoch: 9, Loss: Value(data=0.7220719378115273)
Epoch: 10, Loss: Value(data=0.5013577040845125)
Epoch: 11, Loss: Value(data=0.37121601786125785)
Epoch: 12, Loss: Value(data=0.290840156958421)
Epoch: 13, Loss: Value(data=0.2375133056552385)
Epoch: 14, Loss: Value(data=0.19996046197292663)
Epoch: 15, Loss: Value(data=0.17225373199557326)
Epoch: 16, Loss: Value(data=0.15104137465003115)
Epoch: 17, Loss: Value(data=0.1343138022267933)
Epoch: 18, Loss: Value(data=0.12080305385143722)
Epoch: 19, Loss: Value(data=0.10967348329279518)


In [8]:
ypred = [n(x) for x in xs]
ypred

[Value(data=0.8885797465115218),
 Value(data=-0.8459486721432505),
 Value(data=-0.8059148532670184),
 Value(data=0.8370942843524493)]